<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np 
import pandas as pd
import random 
import re
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences


from tensorflow.keras.layers import Dense, Embedding, Flatten ,MultiHeadAttention,Dropout
from tensorflow.keras import Sequential

import matplotlib.pyplot as plt

In [19]:
## 1) Sentences will be tokenized using the TextVectorization 
## 2) 12.5 % of the tokens will be replaced with the [MASK] token. Substituded tokens will be 
## chosen randomly. To select the random tokens, random sampling will be chosen 

## 3) segment id will be determined at the second step
## 4) positional embeddings will be directly taken from the article

In [20]:
max_length =  128  ## 256
corpus_length = 128   ## 30000
embedding_dimension = 128   ## 128
vocab_size = 128  ## 30000
number_heads = 8



In [21]:
trial_sentences = [
    'my name is john',
    'london is the capital is the england',
    'weather is cold at the northern hemisphere of the world',
    'today, i will go the school'
]

longest_sentence = []
for i in trial_sentences:
    longest_sentence.append(len(i.split()))
    
print(np.max(longest_sentence))

longest_sentence = np.max(longest_sentence)

10


In [22]:
def masking_procedure (text,tok):
    
    # tf_tokenizer = Tokenizer()
    tf_tokenizer = tok
    tf_tokens=tf_tokenizer.fit_on_texts(text)

    corpus = tf_tokenizer.word_index  ### TOKEN SET WITH TOKEN IDS WITHOUT THE SPECIAL TOKENS 
    extended_corpus = corpus.copy()   ### TOKEN SET WITH SPECIAL TOKENS

    extended_corpus['MASK'] = len(corpus)+1

    corpus_tokens = []   #### TOKEN SET WITHOUT THE SPECIAL TOKENS
    for i,j in enumerate(corpus):
        corpus_tokens.append(j)
#         print(i,j)

        
    extended_tokens = []
    for i,j in  enumerate(extended_corpus):
    #     print(i,j)
        extended_tokens.append(j)
    # print('maskelenmis_corpus')
    # print('-------')
    subs_tokens=random.sample(corpus_tokens,int(len(corpus_tokens)*0.12)) ## RANDOMLY SELECTED TOKENS TO BE MASKED
#     print(subs_tokens)

    sequences = tf_tokenizer.texts_to_sequences(text)
    padded_sequences= pad_sequences(sequences)
    padded_sequences

    substitute_padded_sequences= padded_sequences.copy()
    actual_padded_sequences = padded_sequences.copy()

    subs_tokens_ids = [ ]  ## ID OF THE SUBS TOKENS

    for i in subs_tokens:
        subs_tokens_ids.append(tf_tokenizer.word_index[i])
    
    
    mask_token_id = 20
#     print(subs_tokens_ids)



    for i in range(len(extended_tokens)):
        if i == (subs_tokens_ids[0]-1   ):
            extended_tokens[i] = '[MASK]'
    
        if i == (subs_tokens_ids[1]-1):
            extended_tokens[i] = '[MASK]'
        
        
        
    maskeli = []
    for i in range(substitute_padded_sequences.shape[0]):
        for j in range(substitute_padded_sequences.shape[1]):
            if substitute_padded_sequences[i][j] != 0:
                x = (substitute_padded_sequences[i][j]  )
                maskeli.append("".join(extended_tokens[x-1]))
                
                
    maskeli_2 = []
    for i in (substitute_padded_sequences):
        maskeli_2.append('\n')
        for j in range(len(i)):
        
            if (i[j] !=0):
#             print(i[j])
                maskeli_2.append("".join(extended_tokens[i[j] -1]))
    
    
    y= " ".join(maskeli_2)

    y = y.split('\n')
    y = np.delete(y,0,0)
    y
     
    segment_ids = substitute_padded_sequences.copy()

    for i in range(segment_ids.shape[0]):
        for j in range(segment_ids.shape[1]):
            if (segment_ids[i][j] != 0):
                segment_ids[i][j] = 1
    segment_ids = np.array(segment_ids)
    
    return (y , extended_tokens, segment_ids,substitute_padded_sequences,actual_padded_sequences)

In [23]:
tf_tokenizer = Tokenizer()

In [24]:
masked_text, masked_tokens,segment_ids , masked_sequence, original_sequences = masking_procedure(trial_sentences,tf_tokenizer)

In [25]:
masked_sequence,original_sequences,masked_text

(array([[ 0,  0,  0,  0,  0,  0,  3,  4,  2,  5],
        [ 0,  0,  0,  6,  2,  1,  7,  2,  1,  8],
        [ 9,  2, 10, 11,  1, 12, 13, 14,  1, 15],
        [ 0,  0,  0,  0, 16, 17, 18, 19,  1, 20]], dtype=int32),
 array([[ 0,  0,  0,  0,  0,  0,  3,  4,  2,  5],
        [ 0,  0,  0,  6,  2,  1,  7,  2,  1,  8],
        [ 9,  2, 10, 11,  1, 12, 13, 14,  1, 15],
        [ 0,  0,  0,  0, 16, 17, 18, 19,  1, 20]], dtype=int32),
 array([' my [MASK] is john ', ' london is the capital is the england ',
        ' weather is cold at the northern hemisphere of the world ',
        ' [MASK] i will go the school'], dtype='<U57'))

In [26]:
MAX_LEN = 256
EMBED_DIM = 128



def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc


positional_embeddings = get_pos_encoding_matrix(MAX_LEN, EMBED_DIM)

In [27]:
pos_enc = get_pos_encoding_matrix(MAX_LEN, EMBED_DIM)
# pos_enc + segment_ids

total_encoding = masked_sequence+segment_ids # +pos_enc  (bi sekilde positional encodingi sokmam gerek )

In [28]:
total_encoding.shape
intro_shape = total_encoding.shape
masked_sequence = masked_sequence.astype(dtype='float32')


In [36]:
def encoder_structure (masked_sequence):
  emb_layer = layers.Embedding(30000, EMBED_DIM,input_length=max_length)(masked_sequence)
  att_layer = layers.MultiHeadAttention(num_heads=8,key_dim=EMBED_DIM)(emb_layer,emb_layer,emb_layer)
  drop_layer = layers.Dropout(0.1)(att_layer)
  norm_layer = layers.LayerNormalization(epsilon = 1e-6)(emb_layer + drop_layer)
  ff1_layer = layers.Dense(EMBED_DIM , activation='relu')(norm_layer)


  #bu kisimda bir noktada activasyon fonksiyonu olmadan sadece embedding layeri kadar neurona sahip
  #baska bir layer var onu da eklemem gerebilir
  #AYRICA FF1_LAYER'IN CIKIS KISMI SHAPELERINE DIKKAT ET

  return (ff1_layer)


decision_model = tf.keras.Sequential([
                                      
                                      tf.keras.layers.Dense(10,activation = 'softmax'),
                                      tf.keras.layers.Dense(1)
])

decision_model.compile(
    loss= 'categorical_crossentropy',
    optimizer =  'Adam',
    metrics = ['accuracy']
)
decision_input = encoder_structure(masked_sequence)
decision_model.fit(decision_input, masked_sequence, epochs=3)

# BU NOKTADAN SONRA SOFTMAX FONKSIYONU KULLANILARAK OLUSTURULAN MODEL TRAIN EDILECEK 
# INPUT OLARAK ENCODER_STRUCTURE FONKIYONUN 



Epoch 1/3
1/1 [==============================] - 1s 531ms/step - loss: 190.7881 - accuracy: 0.3250
Epoch 2/3
1/1 [==============================] - 0s 6ms/step - loss: 136.6859 - accuracy: 0.3250
Epoch 3/3
1/1 [==============================] - 0s 5ms/step - loss: 130.5456 - accuracy: 0.3250


In [30]:
emb_layer = layers.Embedding(30000, EMBED_DIM,input_length=max_length)(masked_sequence)
att_layer = layers.MultiHeadAttention(num_heads=8,key_dim=EMBED_DIM)(emb_layer,emb_layer,emb_layer)
drop_layer = layers.Dropout(0.1)(att_layer)
norm_layer = layers.LayerNormalization(epsilon = 1e-6)(emb_layer + drop_layer)
ff1_layer = layers.Dense(EMBED_DIM , activation='relu')(norm_layer)


#### FF NN

ff_model = tf.keras.Sequential([
                                tf.keras.layers.Dense(EMBED_DIM, activation= 'relu'),
                                tf.keras.layers.Dense(1)

])



# ff_model.compile(
#     loss = 'categorical_crossentropy',
#     optimizer = 'Adam',
#     metrics = ['accuracy']
# )

# ff_model.fit(norm_layer, masked_sequence, epochs =50)
print('norm_layer_shape',norm_layer.shape)
print('masked_sequence_shape',masked_sequence.shape)
print('encoder_structure_out_shape',ff1_layer.shape)

norm_layer_shape (4, 10, 128)
masked_sequence_shape (4, 10)
encoder_structure_out_shape (4, 10, 128)


In [31]:
print('dense_layer_out', dense_1.shape)
print('attention_layer_out',attention_layer.shape)
print('softmax_layer_out',softmax_layer.shape)
print('reduction_layer_out', reduction_layer.shape)

NameError: ignored

In [ ]:
plt.plot(dense_1[0][1])
plt.plot(attention_layer[0][1])

In [ ]:
# input_layer = layers.Input(shape = (10,))
# l1_layer = layers.Embedding(corpus_length, 10, input_length =10)(input_layer)
# # l1_layer = layers.Flatten()(l1_layer)
# out_layer  = layers.Dense(10, activation= 'softmax')(l1_layer)
# tf_trial_model  = keras.Model(inputs= input_layer, outputs = out_layer)

# tf_trial_model.compile(
#     loss = 'categorical_crossentropy',
#     optimizer = 'Adam',
#     metrics = ['accuracy']
# )

# tf_trial_model.fit(
#                   #  np.transpose(masked_sequence),
#                    masked_sequence, 
#                    original_sequences, 
#                    epochs = 100)